In [4]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader
from torchvision import transforms

from transformers import AdamW, get_scheduler

from jre_utils.datapath import (
    processed_pretraining_data_paths,
    pretraining_data_paths,
    pretrained_weights_paths,
)

from jre_utils.data import SKTimeSeriesDataset, PadAndMask, ToNumpy, ToTensor
from jre_utils.models import TimeSeriesTransformerModel
from jre_utils.engine import (
    evaluate,
    train,
    EarlyStopper,
)
from jre_utils.metrics import MSELossWeighted

from sktime.datasets import load_UCR_UEA_dataset, load_forecastingdata

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [5]:
col_rename_dict = {
    "ArrowHead": {
        "level_0": "id",
        "timepoints": "time_idx",
        "dim_0": "value",
        "class_val": "class",
    }
}

In [6]:
dataset = 'ArrowHead'
df = load_UCR_UEA_dataset(name=dataset, return_X_y=False, return_type="pd-multiindex")
df = df.reset_index()
df = df.rename(columns = col_rename_dict[dataset])
df = df.astype(float)

In [7]:
# Done: Convert all dataframes into id, time_idx, value, classVal (optional)

In [8]:
eval_start_time = df["time_idx"].quantile(0.9)
train_df = df[df["time_idx"] <= eval_start_time]
eval_df = df[df["time_idx"] > eval_start_time]


In [15]:
BATCH_SIZE = 64
window_length = 10
shift = 1

metrics = ["value", "class"]
feature_columns = ["value", "class"]

train_dataset = SKTimeSeriesDataset(
    df,
    train_df,
    metrics=metrics,
    feature_columns=feature_columns,
    window_length=window_length,
    shift=shift,
    transform=transforms.Compose(
        [ToNumpy(), PadAndMask(pad_length=window_length), ToTensor()]
    ),
)

eval_dataset = SKTimeSeriesDataset(
    df,
    eval_df,
    metrics=metrics,
    feature_columns=feature_columns,
    window_length=window_length,
    shift=shift,
    transform=transforms.Compose(
        [ToNumpy(), PadAndMask(pad_length=window_length), ToTensor()]
    ),
)

In [17]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
eval_dataloader = DataLoader(eval_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [18]:
# Build time series transfomer classifier
for i, sample in enumerate(train_dataloader):
    if i == 0:
        print(i, sample["window"].shape, sample["target"].shape)
        break

0 torch.Size([64, 10, 2]) torch.Size([64, 2])


In [19]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# device = "cpu"
print(f"Using {device} device")

Using mps device


In [20]:
n_features = len(feature_columns)
target_size = len(metrics)
d_model = 128
d_hid = 256
nlayers = 4
nhead = 4
dropout = 0.1
enc_dropout = 0

model = TimeSeriesTransformerModel(
    n_features=n_features,
    target_size=target_size,
    d_model=d_model,
    nhead=nhead,
    d_hid=d_hid,
    nlayers=nlayers,
    dropout=dropout,
    device=device,
)
model = model.to(device)


In [23]:
from torch.nn import MSELoss

model.eval()
mse_loss = MSELoss().to(device)

with torch.no_grad():
    for batch in eval_dataloader:
        window = batch["window"].to(device)
        mask = batch["mask"].to(device)
        target = batch["target"].to(device)
        weight = batch["weight"].to(device)

        outputs = model(window, mask)
        loss = mse_loss(outputs, target)

        print(
            batch["window"].shape,
            batch["mask"].shape,
            batch["target"].shape,
            batch["weight"].shape,
        )

        print(outputs.shape)
        print("Loss:", loss.item())

        if i == 0:
            break

torch.Size([64, 10, 2]) torch.Size([64, 10]) torch.Size([64, 2]) torch.Size([64, 1])
torch.Size([64, 2])
Loss: 1.6841917037963867
